In [1]:
#constants

DATA_PATH = 'C:\\Users\\gt\\End to End Data Science Project\\Processed data\\cleaned_data.pkl'

ROLE_COLS = ['DevType']
TECH_COLS = ['LanguageHaveWorkedWith',
             'DatabaseHaveWorkedWith',
             'PlatformHaveWorkedWith',
             'WebframeHaveWorkedWith', 
             'MiscTechHaveWorkedWith', 
             'ToolsTechHaveWorkedWith', 
             'NEWCollabToolsHaveWorkedWith']

MLFLOW_TRACKING_URI = 'C:\\Users\\gt\\End to End Data Science Project\\models\\mlruns'
MLFLOW_EXPERIMENT_NAME = "skills_jobs_Stackoverflow"

LOG_PATH = "C:\\Users\\gt\\End to End Data Science Project\\models\\temp\\"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [2]:
#Load pachages
import pandas as pd 
import numpy as np
import logging
import pickle
import random
import plotly 
import os
from pathlib import Path

import mlflow
from mlflow.tracking import MlflowClient


from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression

from matplotlib import pyplot as plt

<hr>

## Functions

In [3]:
def calculate_quality(ground_truth, predictions, metric_function, sort_values=False):
    quality_scores = {}
    for col in predictions.columns:
        role_pred  = predictions[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred) * 100, 2)
        
    quality_scores = pd.Series(quality_scores.values(), index=quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    
    return quality_scores

<hr>

## Initialize

### Create directories


In [4]:
#exist_ok: where the command does nothing and does not raise an error if the directory already exists
Path(MLFLOW_TRACKING_URI).mkdir(parents=True, exist_ok=True)
Path(LOG_PATH).mkdir(parents=True, exist_ok=True)

## Read data


In [5]:
# Read Data
df = pd.read_pickle(DATA_PATH)
df.head()

DevType                           \
  Academic researcher Data or business analyst   
0                   0                        0   
3                   0                        0   
4                   0                        0   
8                   0                        0   
9                   0                        0   

                                                                        \
  Data scientist or machine learning specialist Database administrator   
0                                             0                      0   
3                                             0                      0   
4                                             0                      0   
8                                             0                      0   
9                                             1                      0   

                                                               \
  DevOps specialist Developer, QA or test Developer, back-end   
0                 0                     0                   0   
3                 0                     0                   0   
4                 0                     0                   0   
8                 0                     0                   0   
9                 0                     0                   0   

                                                 \
  Developer, desktop or enterprise applications   
0                                             0   
3                                             0   
4                                             1   
8                                             0   
9                                             0   

                                                                    ...  \
  Developer, embedded applications or devices Developer, front-end  ...   
0                                           0                    0  ...   
3                                           0                    1  ...   
4                                           0                    0  ...   
8                                           0                    1  ...   
9                                           0                    0  ...   

  skills_clusters                                                \
  skills_group_10 skills_group_11 skills_group_2 skills_group_3   
0               0               0              2              0   
3               0               0              4              0   
4               0               0              2              1   
8               0               0              2              0   
9               0               0              1              0   

                                                                              \
  skills_group_4 skills_group_5 skills_group_6 skills_group_7 skills_group_8   
0              0              0              4              0              4   
3              0              0              0              0              0   
4              0              1              0              2              1   
8              0              1              0              0              0   
9              0              0              0              1              0   

                  
  skills_group_9  
0              0  
3              0  
4              4  
8              0  
9              0  

[5 rows x 153 columns]

## Balance classes


In [6]:
# Check the total samples of roles
roles_df = df["DevType"].copy()
roles_df.sum(axis=0).sort_values(ascending=False)

Developer, full-stack                            20655
Developer, back-end                              17084
Developer, front-end                              8932
Developer, desktop or enterprise applications     4845
Developer, mobile                                 4751
DevOps specialist                                 3056
Data scientist or machine learning specialist     2460
Developer, embedded applications or devices       2138
System administrator                              2069
Engineer, data                                    1941
Academic researcher                               1708
Data or business analyst                          1658
Database administrator                            1210
Developer, QA or test                             1135
Scientist                                         1046
Developer, game or graphics                        899
dtype: int64

In [7]:
# Resample roles
samples_per_class = 1200
resampled_roles = []

for role in roles_df.columns:
    
    sub_df =roles_df.loc[roles_df[role] == 1].copy()
    
    if len(sub_df) < samples_per_class:
        # upper sampling: increasing the number of samples in this class by sampling with replacement
        sub_df = sub_df.sample(samples_per_class, replace = True, random_state=0)
    else: 
        # down sampling   
        sub_df = sub_df.sample(samples_per_class, random_state=0) #replace = False by default

    resampled_roles.append(sub_df)   
    
# Construct dfs
roles_df=pd.concat(resampled_roles)
df = df.loc[roles_df.index].copy()

In [8]:
# Check the total samples of roles
roles_df.sum(axis=0)

Academic researcher                              2280
Data or business analyst                         1965
Data scientist or machine learning specialist    2576
Database administrator                           1765
DevOps specialist                                2170
Developer, QA or test                            1514
Developer, back-end                              5710
Developer, desktop or enterprise applications    2690
Developer, embedded applications or devices      1773
Developer, front-end                             2614
Developer, full-stack                            5602
Developer, game or graphics                      1441
Developer, mobile                                2155
Engineer, data                                   2046
Scientist                                        1910
System administrator                             2110
dtype: int64

## Split

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop("DevType", axis=1), 
                                                    df["DevType"], 
                                                    random_state=0)

D:\Anaconda\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


<hr>

## Train models


### Initialize MLflow

In [10]:
# Initialize client and experiment
client = MlflowClient() 
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)
exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)

2022/07/30 08:49:45 INFO mlflow.tracking.fluent: Experiment with name 'skills_jobs_Stackoverflow' does not exist. Creating a new experiment.


## 1. Logistic regression


**Multi target classification** This strategy consists of fitting one classifier per target. This is a simple strategy for extending classifiers that do not natively support multi-target classification.

In [11]:
clf = make_pipeline(StandardScaler(),
                    MultiOutputClassifier(LogisticRegression()))

clf.fit(X_train.values, Y_train.values)
predictions =  pd.DataFrame(clf.predict(X_train.values),
                            columns=Y_train.columns)

In [12]:
# Evaluate on training set
predictions =  pd.DataFrame(clf.predict(X_train.values),
                            columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

In [13]:
# Evaluate on test set
predictions =  pd.DataFrame(clf.predict(X_test.values), columns=Y_test.columns)
test_scores = {score.__name__: calculate_quality(Y_test, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
test_scores = pd.concat(test_scores,axis=1)
mean_test_scores = test_scores.mean()

In [14]:
print(mean_test_scores)
test_scores

accuracy_score     88.939375
precision_score    62.420625
recall_score       29.890000
f1_score           38.613125
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
Academic researcher,90.02,64.42,30.94,41.80
Data or business analyst,91.12,65.99,26.58,37.90
Data scientist or machine learning specialist,89.96,67.58,49.23,56.96
Database administrator,90.90,56.10,5.20,9.52
DevOps specialist,90.21,65.38,25.47,36.66
"Developer, QA or test",91.71,28.57,0.51,1.00
"Developer, back-end",74.40,62.64,35.72,45.50
"Developer, desktop or enterprise applications",87.52,61.58,18.20,28.09
"Developer, embedded applications or devices",92.65,64.79,33.17,43.88
"Developer, front-end",88.40,67.91,36.76,47.70


The precision_score is not good so, we need to use more complicated model to get better precision_score

## Log run


### 1. Prepare


In [15]:
# Data details
data_details = {"data_path": DATA_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    Y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [16]:
# Model
model = {"model_description": "Baseline model: Logistic Regression ",
         "model_details": str(clf),
         "model_object": clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [17]:
print(str(clf))

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('multioutputclassifier',
                 MultiOutputClassifier(estimator=LogisticRegression()))])


In [18]:
# Performance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

## 2. Log


In [19]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, run_name=model["model_description"]):
    # Log pickles 
    mlflow.log_artifacts(LOG_PATH)
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 
    

2022/07/30 08:49:49 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



In [20]:
exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
# Get all runs 
runs = mlflow.search_runs([exp.experiment_id])
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.precision_score,metrics.f1_score,metrics.accuracy_score,metrics.recall_score,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.source.type
0,bb29a0874d724f42b4b0ee1ffbb25f01,2,FINISHED,file:///C:/Users/gt/End%20to%20End%20Data%20Sc...,2022-07-30 06:49:49.326000+00:00,2022-07-30 06:49:49.447000+00:00,62.420625,38.613125,88.939375,29.89,Baseline model: Logistic Regression,gt,D:\Anaconda\lib\site-packages\ipykernel_launch...,LOCAL
